# Energy Estimator Tool
## Introduction
This is a port of the enVerid COVID-19 Energy Estimator 2 Excel Spreadsheet to a Jupyter Notebook to better understand the calculation methods the spreadsheet uses.

The spreadsheet must be present in the same folder folder as this file for the reference tables to be read.

### Dependencies
- [pandas](https://pypi.org/project/pandas/)
- [openpyxl](https://pypi.org/project/openpyxl/)
- [ipywidgets](https://pypi.org/project/ipywidgets/)
- [tabulate](https://pypi.org/project/tabulate/)

## Reference Tables
The same reference tables are used in this notebook.

The following code imports the reference tables. Run it once before any other cell.

In [ ]:
import pandas as pd

SPREADSHEET_PATH = "enVerid COVID-19 Energy Estimator 2.xlsx"

filters = pd.read_excel(
    SPREADSHEET_PATH, "Table 2 - Filtration Info", skiprows=2, skipfooter=4, index_col=1
).dropna(axis="columns")

# ASHRAE 62.1 2016 Outdoor Air rates (ref. table 6-1)
# TODO: update to ASHRAE 62.1 2022
oa_rates = pd.read_excel(
    SPREADSHEET_PATH,
    "Table 3 - ASHRAE 62.1 OA Rates",
    header=1,
    index_col=0,
    names=["RPeople", "RArea"],
)

# TODO: replace this with custom data for any city
operation_info_126 = pd.read_excel(
    SPREADSHEET_PATH, "Table 1 - Operational Info", header=3, nrows=21, index_col=1
)

operation_info_247 = pd.read_excel(
    SPREADSHEET_PATH, "Table 1 - Operational Info", header=27, nrows=21, index_col=1
)

## General Inputs

In [ ]:
from ipywidgets import widgets, interact
from IPython.display import display, HTML

repr_city = None
space_type = None
floor_area = None  # ft^2
avg_ceil_hgt = None  # ft
occupancy = None  # number occupants
oa_calc_method = None
total_supply_airflow = None  # CFM
vent_efficiency = None


# Display Representative city dropdown widget
def set_repr_city(x):
    global repr_city
    repr_city = x


repr_city_select = interact(set_repr_city, x=operation_info_126.index.values)
repr_city_select.widget.children[0].description = "Representative city"
repr_city_select.widget.children[0].style = {
    "description_width": "initial"
}  # Don't cut off text


# Display Space Type selection dropdown widget
def set_space_type(x="Office space"):
    global space_type
    space_type = x


space_select = interact(set_space_type, x=oa_rates.index.values)
space_select.widget.children[0].description = "Space type"

# Display floor area slider widget
MIN_FLOOR_AREA = 100
MAX_FLOOR_AREA = 100_000
FLOOR_STEP = 100


def set_floor_area(x=50000):
    global floor_area
    floor_area = x


floor_slider = interact(set_floor_area, x=(MIN_FLOOR_AREA, MAX_FLOOR_AREA, FLOOR_STEP))
floor_slider.widget.children[0].description = "Floor area (ft^2)"
floor_slider.widget.children[0].style = {"description_width": "initial"}


# Display average ceiling height slider widget
MIN_CEIL_HT = 7.5
MAX_CEIL_HT = 50
CEIL_STEP = 0.25


def set_ceil_height(x=12):
    global avg_ceil_hgt
    avg_ceil_hgt = x


ceil_ht_slider = interact(set_ceil_height, x=(MIN_CEIL_HT, MAX_CEIL_HT, CEIL_STEP))
ceil_ht_slider.widget.children[0].description = "Avg. ceiling height (ft)"
ceil_ht_slider.widget.children[0].style = {"description_width": "initial"}

# Display occupancy slider widget
MIN_OCC = 1
MAX_OCC = 5000


def set_occupancy(x=250):
    global occupancy
    occupancy = x


occ_slider = interact(set_occupancy, x=(MIN_OCC, MAX_OCC))
occ_slider.widget.children[0].description = "Occupancy (occupants)"
occ_slider.widget.children[0].style = {"description_width": "initial"}


# Display total supply airflow slider widget
MIN_AIRFLOW = 200
MAX_AIRFLOW = 200_000


def set_total_sup_airflow(x=50_000):
    global total_supply_airflow
    total_supply_airflow = x


total_sup_airflow_slider = interact(set_total_sup_airflow, x=(MIN_AIRFLOW, MAX_AIRFLOW))
total_sup_airflow_slider.widget.children[0].description = "Total supply airflow (CFM)"
total_sup_airflow_slider.widget.children[0].style = {"description_width": "initial"}


# Display OA calculation method dropdown widget
def set_ventilation_efficency(x=0.75):
    global vent_efficiency
    vent_efficiency = x


vent_efficiency_slider = interact(set_ventilation_efficency, x=(0, 1.0, 0.01))
vent_efficiency_slider.widget.children[0].description = "System ventilation efficency"
vent_efficiency_slider.widget.children[0].style = {"description_width": "initial"}

In [ ]:
# Display ventilation efficency slider widget
def set_ventilation_calc(x):
    global oa_calc_method
    oa_calc_method = x


CALC_METHODS = ["VRP", "VRP+30%", "IAQP", "100% OA"]
oa_calc_dropdown = interact(set_ventilation_calc, x=CALC_METHODS)
oa_calc_dropdown.widget.children[0].description = "OA Calculation Method"
oa_calc_dropdown.widget.children[0].style = {"description_width": "initial"}

### Results

Calculate required outside airflow

In [ ]:
from tabulate import tabulate


def vrp_calc():
    """Calculate required airflow according to the Ventilation Rate Procedure."""
    occupancy_component = occupancy * oa_rates["RPeople"].loc[space_type]  # CFM
    area_component = floor_area * oa_rates["RArea"].loc[space_type]  # CFM
    return (occupancy_component + area_component) / vent_efficiency  # CFM


if oa_calc_method == "VRP":
    outside_airflow = vrp_calc()  # CFM
elif oa_calc_method == "VRP+30%":
    outside_airflow = vrp_calc() * 1.3
elif oa_calc_method == "IAQP":
    outside_airflow = floor_area * 0.05  # TODO: find out why this 5% factor
elif oa_calc_method == "100% OA":
    outside_airflow = total_supply_airflow
else:
    outside_airflow = int(input("Input a value for outside airflow."))

print(f"Outside airflow: {outside_airflow:.2f} CFM")

Calculate outside air changes per hour

In [ ]:
avg_volume = avg_ceil_hgt * floor_area  # ft^3
outside_air_ach = outside_airflow / avg_volume * 60  # /h
print(f"Ouside air ACH: {outside_air_ach:.2f} /h")

## Outside air ventilation Energy & Operating Costs


In [ ]:
oa_cooling_src = None
oa_heating_src = None

def set_oa_cooling_src(x):
    global oa_cooling_src
    oa_cooling_src = x

def set_oa_heating_src(x):
    global oa_heating_src
    oa_heating_src = x

COOLING_SRCS = ["Electricity"]
oa_cool_dropdown = interact(set_oa_cooling_src, x=COOLING_SRCS)
oa_cool_dropdown.widget.children[0].description = "OA Ventilation Cooling Source"
oa_cool_dropdown.widget.children[0].style = {"description_width": "initial"}

HEATING_SRCS = ["Electricity", "Gas", "Steam"]
oa_heat_dropdown = interact(set_oa_heating_src, x=HEATING_SRCS)
oa_heat_dropdown.widget.children[0].description = "OA Ventilation Heating Source"
oa_heat_dropdown.widget.children[0].style = {"description_width": "initial"}


In [ ]:
hrs_per_day_bldg_operation = None
days_per_wk_bldg_operation = None

def set_hrs_per_day(x=12):
    global hrs_per_day_bldg_operation
    hrs_per_day_bldg_operation = x


def set_days_per_week(x=6):
    global days_per_wk_bldg_operation
    days_per_wk_bldg_operation = x

hrs_per_day_slider = interact(set_hrs_per_day, x=(8, 24))
hrs_per_day_slider.widget.children[0].description = "Hours per day of building operation"
hrs_per_day_slider.widget.children[0].style = {"description_width": "initial"}

days_per_wk_slider = interact(set_days_per_week, x=(1,7))
days_per_wk_slider.widget.children[0].description = "Days per week of building operation"
days_per_wk_slider.widget.children[0].style = {"description_width": "initial"}

In [ ]:
COP = 3
HEATING_EFFICIENCY = 1.00

operating_hrs_per_wk = hrs_per_day_bldg_operation * days_per_wk_bldg_operation
elec_rate = operation_info_126["Estimated Blended Electricity Rate ($/kWh)"].loc[
    repr_city
]

if oa_heating_src == "Electricity":
    heating_energy_rate = elec_rate
    unit = "$/kWh (blended)"
elif oa_heating_src == "Gas":
    heating_energy_rate = operation_info_126["Gas Rate ($/therm)"].loc[repr_city]
    unit = "$/therm"
elif oa_heating_src == "Steam":
    heating_energy_rate = operation_info_126["Steam Rate ($/Mlb)"].loc[repr_city]
    unit = "$/mmBTU"

disp_table = [
    ["Electricity rate", elec_rate, "$/kWh (blended)"],
    ["Heating energy rate", heating_energy_rate, unit],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Energy Recovery Inputs (optional)

In [ ]:
energy_recovery = False


if energy_recovery:
    # TODO: sliders that only appear if checkbox for energy recovery ticked
    summer_recovery_eff = float(input("Summer energy recovery effectiveness (0-1.0): "))
    winter_recovery_eff = float(input("Winter energy recovery effectivness (0-1.0): "))
else:
    summer_recovery_eff = 0
    winter_recovery_eff = 0

## Indoor Air Filtration Operating Cost & First Cost Inputs

In [ ]:
RECIRC_FILTRATION = True
MERV = "MERV 11"  # TODO: dropdown for filter types

avg_press_drop = filters["average pressure drop (in.w.c.)"].loc[MERV]  # in. wc
fan_eff = 0.85
mtr_eff = 0.85
max_filter_airspeed = 500  # fpm
est_filter_lifespan = filters["filter lifespan (months)"].loc[MERV]  # months
filter_replace_labor_cost = 5  # $/filter
filtration_ach = 5  # /hr

## Air Cleaner Operating Cost & First Cost Inputs

In [ ]:
AIR_CLEANERS = False


air_cleaner_quant = 50
air_cleaner_supply_air = 200  # CFM per air cleaner
air_cleaner_cadr = 200  # CFM per air cleaner

additional_ach = air_cleaner_cadr * air_cleaner_quant / avg_volume * 60

cleaner_first_cost = 2200  # $ per air cleaner. Includes hardware + install cost
air_cleaner_pwr = air_cleaner_supply_air * 0.0003  # kW per air cleaner
cleaner_est_lifetime_before_maint = 12  # months
cleaner_maint_labor_cost = 50  # $ per unit
cleaner_maint_material_cost = 150  # $ per unit

## Outside Air Ventilation Energy Consumption & Cost

In [ ]:
if operating_hrs_per_wk < (24 * 7):
    ops_info = operation_info_126
    op_hours_baseline = 72  # Operations info based on operating 12/6
else:
    ops_info = operation_info_247
    op_hours_baseline = 168  # Operations info based on operating 24/7

cooling_oa_ventilation_energy = (
    outside_airflow
    * ops_info["Cooling energy (kWh/cfm)"].loc[repr_city]
    * (1 - summer_recovery_eff)
    / (COP / 3)
    * (operating_hrs_per_wk / op_hours_baseline)
)  # kWh / yr

oa_cooling_cost = cooling_oa_ventilation_energy * elec_rate  # $/yr

if oa_heating_src == "Electricity":
    heating_energy = ops_info["Heating energy (kWh/cfm)"]
    unit = "kWh/yr"
elif oa_heating_src == "Steam":
    heating_energy = ops_info["Heating energy (mmBTU/cfm)"]
    unit = "mmBTU/yr"
elif oa_heating_src == "Gas":
    heating_energy = ops_info["Heating energy (therms/cfm)"]
    unit = "therm/yr"

heating_oa_ventilation_energy = (
    outside_airflow
    * heating_energy.loc[repr_city]
    * (1 - winter_recovery_eff)
    / HEATING_EFFICIENCY
    * (operating_hrs_per_wk / op_hours_baseline)
)  # (kWh | mmBTU | therm)/yr

oa_heating_cost = heating_oa_ventilation_energy * heating_energy_rate  # $/yr

disp_table = [
    ["Cooling Outside Air Ventilation Energy", cooling_oa_ventilation_energy, "kWh/yr"],
    ["Cooling Outside Air Energy Cost", oa_cooling_cost, "$/yr"],
    ["Heating Outside Air Ventilation Energy", heating_oa_ventilation_energy, unit],
    ["Heating Outside Air Energy Cost", oa_heating_cost, "$/yr"],
]

tabulate(disp_table, floatfmt=",.2f", tablefmt="html")

## Indoor Air Filtration Energy Consumption and Cost

In [ ]:
from math import ceil

CUBIC_METER_PER_HOUR_PER_CFM = 1.699
PASCAL_PER_INCH_WATER = 248.84
METER_PER_SECOND_PER_FPM = 1 / 196.85
SQ_FT_PER_SQ_METER = 10.7639

# Determine cost of running the fan to filter air
if not RECIRC_FILTRATION:
    recirc_airflow = 0
    indoor_air_filt_fan_load = 0  # kW
else:
    recirc_airflow = total_supply_airflow - outside_airflow  # CFM
    recirc_airflow_m3_per_h = recirc_airflow * CUBIC_METER_PER_HOUR_PER_CFM  # m^3/h
    recirc_airflow_m3_per_s = recirc_airflow_m3_per_h / 60 / 60  # m^3/s

    avg_press_drop_pa = avg_press_drop * PASCAL_PER_INCH_WATER  # Pa
    fan_load = recirc_airflow_m3_per_s * avg_press_drop_pa / (fan_eff * mtr_eff)  # W
    indoor_air_filt_fan_load = fan_load / 1000  # kW

annual_filtration_energy = indoor_air_filt_fan_load * operating_hrs_per_wk * 52  # kW/yr
annual_filt_energy_cost = annual_filtration_energy * elec_rate  # $/yr

# Determine the cost of replacing filters
baseline_filter_changes_per_year = 12 / filters["filter lifespan (months)"].loc[MERV]
max_filter_airspeed_m_per_s = max_filter_airspeed * METER_PER_SECOND_PER_FPM  # m/s
min_filter_area_m2 = recirc_airflow_m3_per_s / max_filter_airspeed_m_per_s  # m^2
min_filter_area = ceil(min_filter_area_m2 * SQ_FT_PER_SQ_METER)  # ft^2
material_cost_filt_replacement = (
    baseline_filter_changes_per_year * filters["cost"].loc[MERV] * min_filter_area
)  # $/yr
# TODO: understand this calculation. Is the filter cost the cost per square foot?
# TODO: understand https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7127325/. Cost per unit removal rate?
labor_cost_filt_replacement = (
    baseline_filter_changes_per_year * filter_replace_labor_cost * min_filter_area
)

annual_filter_cost = (
    annual_filt_energy_cost
    + material_cost_filt_replacement
    + labor_cost_filt_replacement
)

disp_table = [
    ["Indoor air filtration fan energy load", indoor_air_filt_fan_load, "kW"],
    ["Annual fan filtration energy", annual_filtration_energy, "kW"],
    ["Material cost of filter replacement", material_cost_filt_replacement, "$/yr"],
    ["Labor cost of filter replacement", labor_cost_filt_replacement, "$/yr"],
    [
        "Annual filter fan energy and filter replacement costs",
        annual_filter_cost,
        "$/yr",
    ],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Air Cleaner Operating Cost & First Cost Inputs

In [ ]:
if AIR_CLEANERS:
    air_cleaner_annual_energy = (
        air_cleaner_pwr * air_cleaner_quant * operating_hrs_per_wk * 52
    )  # kW / yr

    air_cleaner_annual_energy_cost = air_cleaner_annual_energy * elec_rate
    air_cleaner_filter_matrl_cost = (
        cleaner_maint_material_cost
        * air_cleaner_quant
        * 12
        / cleaner_est_lifetime_before_maint
    )

    air_cleaner_maint_labor_cost = (
        cleaner_maint_labor_cost
        * air_cleaner_quant
        * 12
        / cleaner_est_lifetime_before_maint
    )

    annual_air_cleaner_total_cost = (
        air_cleaner_annual_energy_cost
        + air_cleaner_filter_matrl_cost
        + air_cleaner_filter_matrl_cost
    )

    air_cleaner_total_first_cost = cleaner_first_cost * air_cleaner_quant

    disp_table = [
        ["Air cleaner annual energy consumption", air_cleaner_annual_energy, "kW/yr"],
        ["Air cleaner annual energy cost", air_cleaner_annual_energy_cost, "$/yr"],
        ["Air cleaner filter material cost", air_cleaner_filter_matrl_cost, "$/yr"],
        ["Air cleaner maintenance labor cost", air_cleaner_maint_labor_cost, "$/yr"],
        ["Air cleaner total annual cost", annual_air_cleaner_total_cost, "$/yr"],
        ["Air cleaner first cost", air_cleaner_total_first_cost, "$/yr"],
    ]
else:
    air_cleaner_annual_energy = 0
    annual_air_cleaner_total_cost = 0

## Effective Air Changes per Hour

In [ ]:
effective_ach = outside_air_ach + filtration_ach + additional_ach # /h

disp_table = [
    ["Outside air ACH", outside_air_ach, "/h"],
    ["Filtration ACH", filtration_ach, "/h"],
    ["Additional ACH", additional_ach, "/h"],
    ["Effective ACH", effective_ach, "/h"]
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Annual Cost Summary

In [ ]:
annual_oa_vent_energy_cost = oa_cooling_cost + oa_heating_cost
total_annual_cost = (
    annual_filt_energy_cost + annual_filter_cost + annual_air_cleaner_total_cost
)

disp_table = [
    ["Annual outside air ventilation energy cost", annual_oa_vent_energy_cost, "$/yr"],
    ["Annual indoor air filter fan energy and filter replacement costs", annual_filter_cost, "$/yr"],
    ["Annual air cleaner energy consumption and maintenance costs", annual_air_cleaner_total_cost, "$/yr"],
    ["Total annual costs", total_annual_cost, "$/yr"]
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")

## Annual Carbon Emissions

In [ ]:
# NOTE: The enVerid spreadsheet makes a mistake here!
# The emission factor they use is 7.09E-4 t CO2 / kWh.
# That factor is for electricity reductions!
# We are concerned with emissions associated with consumption, not reduction.
# See https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references

ELECTRICITY_EMISSION_FACTOR = 4.33e-4  # t CO2e / kWh
NG_EMISSION_FACTOR = 0.0053  # t CO2e / therm
STEAM_EMISSION_FACTOR = 0.053  # t CO2e / mmBTU  (assumes perfect efficiency)

# Carbon emissions in t CO2e / yr
cooling_co2_tons = cooling_oa_ventilation_energy * ELECTRICITY_EMISSION_FACTOR

if oa_heating_src == "Electricity":
    heating_co2_tons = heating_oa_ventilation_energy * ELECTRICITY_EMISSION_FACTOR
elif oa_heating_src == "Gas":
    heating_co2_tons = heating_oa_ventilation_energy * NG_EMISSION_FACTOR
elif oa_heating_src == "Steam":
    heating_co2_tons = heating_oa_ventilation_energy * STEAM_EMISSION_FACTOR

carbon_emiss_oa_vent_energy = cooling_co2_tons + heating_co2_tons
carbon_emiss_filt_fan_energy = annual_filtration_energy * ELECTRICITY_EMISSION_FACTOR
carbon_emiss_air_cleaner_energy = (
    air_cleaner_annual_energy * ELECTRICITY_EMISSION_FACTOR
)

total_carbon_emissions = (
    carbon_emiss_oa_vent_energy
    + carbon_emiss_filt_fan_energy
    + carbon_emiss_air_cleaner_energy
)

disp_table = [
    [
        "Carbon emissions - outside air ventilation energy",
        carbon_emiss_oa_vent_energy,
        "metric tons of CO2/yr",
    ],
    [
        "Carbon emissions - filtration fan energy",
        carbon_emiss_filt_fan_energy,
        "metric tons of CO2/yr",
    ],
    [
        "Carbon emissions - air cleaner energy",
        carbon_emiss_oa_vent_energy,
        "metric tons of CO2/yr",
    ],
    ["Total Carbon Emissions", total_carbon_emissions, "metric tons of CO2/yr"],
]

tabulate(disp_table, floatfmt=".2f", tablefmt="html")